In [3]:
import pandas as pd
import xml.etree.cElementTree as ET
import xml.dom.minidom as md

In [5]:
def initialize_xml(params):
    #basic meta info
    imageloc = params['image location']
    module_no = params['module number']
    timer = params['question time limit']
    author = params['author']
    title = params['title']
    
    #text to display
    covertext = params['cover text']
    introtext_1 = params['intro text 1']
    introtext_2 = params['intro text 2']
    introtext_3 = params['intro text 3']

    #create the xml data
    root = ET.Element("QUIZ")
    ET.SubElement(root,'URLBASE').text = imageloc
    ET.SubElement(root,'NOTRACKMODULE').text = module_no
    ET.SubElement(root,'NOCONFIRM')
    ET.SubElement(root,'TIMER').text = timer
    ET.SubElement(root,'ID').text = module_no
    ET.SubElement(root,'AUTHOR').text = author
    ET.SubElement(root,'TITLE').text = title

    ET.SubElement(root,'COVER').text = covertext

    q = ET.SubElement(root,'QUESTION')
    ET.SubElement(q,'TEXT').text = introtext_1

    q = ET.SubElement(root,'QUESTION')
    ET.SubElement(q,'NOTIMER')
    ET.SubElement(q,'TEXT').text = introtext_2

    q = ET.SubElement(root,'QUESTION')
    ET.SubElement(q,'NOTIMER')
    ET.SubElement(q,'TEXT').text = introtext_3

    return root

In [9]:
def add_qs_from_excel(xmlroot, fname, question_text):
    df = pd.read_excel(fname)
    df['Answer image (optional)'].fillna(df['Image'],inplace=True)
    answer_options = df['Answer choices'].dropna().tolist()

    for i, row in df.iterrows():
        q = ET.SubElement(xmlroot,'QUESTION')
        ET.SubElement(q,'FEEDBACK')
        ET.SubElement(q,'TEXT').text = question_text
        ET.SubElement(q,'IMG').text = row['Image']
        ET.SubElement(q,'ANSIMG').text = row['Answer image (optional)']
        correct_ans = row['Answer']

        for answer in answer_options:
            a = ET.SubElement(q,'ANSWER')
            ET.SubElement(a,'TAG').text = correct_ans
            ET.SubElement(a,'TEXT').text = answer
            if answer == correct_ans:
                ET.SubElement(a,'CORRECT')
    
    return xmlroot

In [8]:
def write_xml(xmlroot, outputfile):
    dom = md.parseString(ET.tostring(xmlroot))
    xml_string = dom.toprettyxml()
    part1, part2 = xml_string.split('?>')
    m_encoding = 'UTF-8'
    fname = "content.xml"
    with open(outputfile, 'w') as xfile:
        xfile.write(part1 + 'encoding=\"{}\"?>\n'.format(m_encoding) + part2)
        xfile.close()

In [10]:
#initialize some stuff for testing. Can use this as a template to create new modules

introtext_1= '''
&lt;H2&gt;Instructions&lt;/H2&gt;

You're going to see a series of abdominal plain films.  Your task is to decide whether there is ileus, obstruction, or neither.
You can ignore the rest (for now), but other incidentals will be noted in the answers if you want to see if you can get those too.'''

introtext_2 = '''
    &lt;p&gt;
    A timer is available in the bottom right hand corner.  You will get a full &lt;b&gt;15 seconds&lt;/b&gt; for each case.  &lt;p&gt;

    You may drag the timer to move it, including moving it off screen to hide it.  &lt;p&gt;

    The timer for each case starts after the image is displayed.&lt;p&gt;'''

introtext_3 = '''
    &lt;p&gt;
    This module should take about 20 minutes.
    &lt;p&gt;
    Ready?  Here we go.  Click "Next>>" to start your first case.'''

params={
    'image location': '/import_test/images',
    'module number': '1111',
    'question time limit': '15',
    'author': 'import test author',
    'title': 'import test title',
    'cover text': 'import test cover text',
    'intro text 1': introtext_1,
    'intro text 2': introtext_2,
    'intro text 3': introtext_3
}

input_excel = './test.xlsx'
question_text = 'Ileus, obstruction, or normal?'
output_file = 'content.xml'

#create the module
xmlroot = initialize_xml(params)
xmlroot = add_qs_from_excel(xmlroot,input_excel,question_text)
write_xml(xmlroot,output_file)


# old stuff for testing

In [52]:
root = ET.Element("QUIZ")

ET.SubElement(root,'URLBASE').text = '/import_test/images'
ET.SubElement(root,'NOTRACKMODULE').text = '1111'
ET.SubElement(root,'NOCONFIRM')
ET.SubElement(root,'TIMER').text = '15'
ET.SubElement(root,'ID').text = '1111'
ET.SubElement(root,'AUTHOR').text = 'import test'
ET.SubElement(root,'ID').text = 'imp'

ET.SubElement(root,'COVER').text = 'Dummy cover text'

q = ET.SubElement(root,'QUESTION')
ET.SubElement(q,'TEXT').text = '''

    &lt;H2&gt;Instructions&lt;/H2&gt;

    You're going to see a series of abdominal plain films.  Your task is to decide whether there is ileus, obstruction, or neither.
    You can ignore the rest (for now), but other incidentals will be noted in the answers if you want to see if you can get those too.'''

q = ET.SubElement(root,'QUESTION')
ET.SubElement(q,'NOTIMER')
ET.SubElement(q,'TEXT').text = '''
    &lt;p&gt;
    A timer is available in the bottom right hand corner.  You will get a full &lt;b&gt;15 seconds&lt;/b&gt; for each case.  &lt;p&gt;

    You may drag the timer to move it, including moving it off screen to hide it.  &lt;p&gt;

    The timer for each case starts after the image is displayed.&lt;p&gt;'''

q = ET.SubElement(root,'QUESTION')
ET.SubElement(q,'NOTIMER')
ET.SubElement(q,'TEXT').text = '''
    &lt;p&gt;
    This module should take about 20 minutes.
    &lt;p&gt;
    Ready?  Here we go.  Click "Next>>" to start your first case.'''

In [54]:
df = pd.read_excel('./test.xlsx')
df['Answer image (optional)'].fillna(df['Image'],inplace=True)
answer_options = df['Answer choices'].dropna().tolist() #['Normal', 'Ileus', 'Obstruction']
question_text = 'Ileus, obstruction, or normal?'

for i, row in df.iterrows():
    q = ET.SubElement(root,'QUESTION')
    ET.SubElement(q,'FEEDBACK')
    ET.SubElement(q,'TEXT').text = question_text
    ET.SubElement(q,'IMG').text = row['Image']
    ET.SubElement(q,'ANSIMG').text = row['Answer image (optional)']
    correct_ans = row['Answer']

    for answer in answer_options:
        a = ET.SubElement(q,'ANSWER')
        ET.SubElement(a,'TAG').text = correct_ans
        ET.SubElement(a,'TEXT').text = answer
        if answer == correct_ans:
            ET.SubElement(a,'CORRECT')

In [55]:
dom = md.parseString(ET.tostring(root))
xml_string = dom.toprettyxml()
part1, part2 = xml_string.split('?>')
m_encoding = 'UTF-8'
fname = "content.xml"
with open(fname, 'w') as xfile:
    xfile.write(part1 + 'encoding=\"{}\"?>\n'.format(m_encoding) + part2)
    xfile.close()
